In [1]:
import numpy as np
import cv2
import glob
import os

In [13]:
# 将图片转化成视频
img_array = []
for filename in glob.glob("C:\\Users\\yeung\\Desktop\\Subject2Activity10Trial1Camera1\\*.png"):
    img = cv2.imread(filename)
    img_array.append(img)

height, width, layers = img.shape
size = (width, height)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("jump.mp4", fourcc, 18, size)
for img in img_array:
    out.write(img)
out.release()

In [2]:
def load_label_map(file_path):
    """Load Label Map.

    Args:
        file_path (str): The file path of label map.

    Returns:
        dict: The label map (int -> label name).
    """
    lines = open(file_path).readlines()
    lines = [x.strip().split(': ') for x in lines]
    return {int(x[0]): x[1] for x in lines}

In [3]:
eld_file_path = "tools/data/ava/label_map.txt"
old_label = load_label_map(eld_file_path)

In [ ]:
def filter_label_map(outputs, label_map):
    """将81类标签聚合成我们的9类标签"""
    file_path = "tools/data/ava/my_label.txt"
    lines = open(file_path).readlines()
    # 用来存储整合后的数据
    res = np.zeros((len(lines)+1))
    lines = [x.strip().split(": ") for x in lines]
    label_map = {}
    for i, label in enumerate(lines):
        label_map[i] =  label[1]
        inds = [int(j) for j in label[0].split("*")]
        for j in inds:
            res[i] += outputs[j]
            outputs[j] = 0
    i += 1
    label_map[i] = 'others'
    res[i] = sum(outputs)
    return res, label_map
    

{1: 'bend/bow (at the waist)',
 3: 'crouch/kneel',
 4: 'dance',
 5: 'fall down',
 6: 'get up',
 7: 'jump/leap',
 8: 'lie/sleep',
 9: 'martial art',
 10: 'run/jog',
 11: 'sit',
 12: 'stand',
 13: 'swim',
 14: 'walk',
 15: 'answer phone',
 17: 'carry/hold (an object)',
 20: 'climb (e.g., a mountain)',
 22: 'close (e.g., a door, a box)',
 24: 'cut',
 26: 'dress/put on clothing',
 27: 'drink',
 28: 'drive (e.g., a car, a truck)',
 29: 'eat',
 30: 'enter',
 34: 'hit (an object)',
 36: 'lift/pick up',
 37: 'listen (e.g., to music)',
 38: 'open (e.g., a window, a car door)',
 41: 'play musical instrument',
 43: 'point to (an object)',
 45: 'pull (an object)',
 46: 'push (an object)',
 47: 'put down',
 48: 'read',
 49: 'ride (e.g., a bike, a car, a horse)',
 51: 'sail boat',
 52: 'shoot',
 54: 'smoke',
 56: 'take a photo',
 57: 'text on/look at a cellphone',
 58: 'throw',
 59: 'touch (an object)',
 60: 'turn (e.g., a screwdriver)',
 61: 'watch (e.g., TV)',
 62: 'work on a computer',
 63: 'writ